In [35]:
import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import CovarianceShrinkage
from pypfopt.expected_returns import mean_historical_return
from sqlalchemy import create_engine
import numpy as np
import cvxpy as cp

In [36]:
engine = create_engine('sqlite:///GDS_IndividualProject.db', echo=False)

df_sample = pd.read_sql_table('sample', con=engine, index_col='Date')
df_mkt_cap = pd.read_sql_table('market_cap', con=engine, index_col='Date')
df_sector = pd.read_sql_table('sectors', con=engine)
df_emissions = pd.read_sql_table('emissions', con=engine)
df_esg = pd.read_sql_table('esg', con=engine)

In [37]:
# df_sample.drop('2017-12', inplace=True) # Returns data so drop 1st row

df_sector = df_sector.sort_values('Instrument')

df_emissions = df_emissions.loc[df_emissions['RIC'].isin(df_sample.columns.to_list())].sort_values('RIC').reset_index(drop=True)
df_esg = df_esg.loc[df_esg['RIC'].isin(df_sample.columns.to_list())].sort_values('RIC').reset_index(drop=True)

df_emission_scores = df_emissions[['RIC', 'Carbon Intensity']].merge(df_esg, on='RIC', how='outer')

In [38]:
for column in df_sample.columns:
    df_sample[[column]] = df_sample[[column]].apply(
        lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True, random_state=123), x))

In [39]:
benchmark_weights = (df_mkt_cap/(df_mkt_cap.sum(axis=1).values.reshape(-1,1))).tail(1).T.rename(columns={'2022-12': 'Weight'})
benchmark_returns = pd.DataFrame((benchmark_weights['Weight'].values*df_sample).sum(axis=1), columns=['Benchmark Returns'])

In [40]:
benchmark_weights

Date,Weight
ABM,0.001120
ABT,0.073595
ACN,0.067542
ADBE.O,0.060149
AFL,0.017197
...,...
WDAY.O,0.016533
WK,0.001696
WMT,0.147007
ZBRA.O,0.005090


In [41]:
mu_benchmark_monthly = benchmark_returns.mean()
mu_benchmark_annual = (1+mu_benchmark_monthly)**12-1

sigma_sq = benchmark_returns.var(ddof=1)
sigma_benchmark_monthly = benchmark_returns.std(ddof=1)
sigma_benchmark_annual = sigma_benchmark_monthly*np.sqrt(12)

sr_benchmark = mu_benchmark_annual/sigma_benchmark_annual

num_inv_benchmark = np.sum(benchmark_weights.values>0)
num_sectors_benchmark = df_sector.merge(benchmark_weights.loc[benchmark_weights['Weight']>0], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

In [42]:
benchmark_emissions = (benchmark_weights['Weight'].values*df_emission_scores.drop(columns=['RIC']).T).sum(1)

In [43]:
benchmark_emissions

Carbon Intensity              47.669643
ESG Score                     73.409004
Social Pillar Score           75.560752
Environmental Pillar Score    68.990455
Governance Pillar Score       73.634034
dtype: float64

#### Mean-variance portfolio

In [44]:
mu = mean_historical_return(df_sample, frequency=12, returns_data=True)
S = risk_models.risk_matrix(df_sample, method='ledoit_wolf', returns_data=True)

In [45]:
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))
weights = ef.max_sharpe()

In [46]:
tangency_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
tangency_weights[tangency_weights['Weight']>0].sort_values(by='Weight', ascending=False)

,Weight
CHK.O,0.271608
OTIS.K,0.219058
BAH,0.105264
PGR,0.103398
CTVA.K,0.089192
PWR,0.061907
RLI,0.036290
ABT,0.036190
WK,0.033117
ETSY.O,0.024484


In [47]:
tangency_returns = (tangency_weights.T.values*df_sample).sum(axis=1)

In [48]:
mu_tangency_monthly = tangency_returns.mean()
mu_tangency_annual = (1+mu_tangency_monthly)**12-1

sigma_sq_tangency = tangency_returns.var(ddof=1)
sigma_tangency_monthly = tangency_returns.std(ddof=1)
sigma_tangency_annual = sigma_tangency_monthly*np.sqrt(12)

sr_tangency = mu_tangency_annual/sigma_tangency_annual

num_inv_tangency = np.sum(tangency_weights.values>0)
num_sectors_tangency = df_sector.merge(tangency_weights.loc[tangency_weights['Weight']>0], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

In [49]:
tangency_emissions = (tangency_weights['Weight'].values*df_emission_scores.drop(columns=['RIC']).T).sum(1)

In [50]:
tangency_emissions

Carbon Intensity              56.885315
ESG Score                     61.358715
Social Pillar Score           63.776742
Environmental Pillar Score    41.449690
Governance Pillar Score       72.547599
dtype: float64

#### Constrained Portfolio - 10% Reduced Emissions

In [51]:
# Define constraints
def TE(w, portfolio_returns, benchmark_returns):
    xi= (w @ portfolio_returns.T - benchmark_returns.values.reshape(-1))
    mean = cp.sum(xi)/len(benchmark_returns)
    return cp.sum_squares(xi- mean)

def CI(w, portfolio_intensity):
    xi = w @ portfolio_intensity.T
    carbon_intensity = cp.sum(xi)
    return carbon_intensity

In [76]:
ef_constrained1 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained1.add_constraint(lambda w: TE(w, df_sample, benchmark_returns) <= 0.1**2)
ef_constrained1.add_constraint(lambda w: CI(w, df_emission_scores['Carbon Intensity']) <= 0.9*benchmark_emissions)
weights = ef_constrained1.convex_objective(TE, portfolio_returns=df_sample, benchmark_returns=benchmark_returns)

In [83]:
constrained1_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
constrained1_weights[constrained1_weights['Weight']>0].sort_values(by='Weight', ascending=False)

,Weight
WMT,0.150205
ACN,0.086685
ABT,0.067828
ADBE.O,0.064399
QCOM.O,0.048857
...,...
INFN.O,0.000548
GOGL.O,0.000533
TPIC.O,0.000437
BOOM.O,0.000366


In [78]:
constrained1_returns = (constrained1_weights.T.values*df_sample).sum(axis=1)

In [84]:
mu_constrained1_monthly = constrained1_returns.mean()
mu_constrained1_annual = (1+mu_constrained1_monthly)**12-1

sigma_sq_constrained1 = constrained1_returns.var(ddof=1)
sigma_constrained1_monthly = constrained1_returns.std(ddof=1)
sigma_constrained1_annual = sigma_constrained1_monthly*np.sqrt(12)

sr_constrained1 = mu_constrained1_annual/sigma_constrained1_annual

num_inv_constrained1 = np.sum(constrained1_weights.values>10**-4)
num_sectors_constrained1 = df_sector.merge(constrained1_weights.loc[constrained1_weights['Weight']>10**-4], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

In [85]:
constrained1_emissions = (constrained1_weights['Weight'].values*df_emission_scores.drop(columns=['RIC']).T).sum(1)

#### Constrained Portfolio - 20% Carbon Intensity Reduction

In [70]:
ef_constrained2 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained2.add_constraint(lambda w: TE(w, df_sample, benchmark_returns) <= 0.1**2)
ef_constrained2.add_constraint(lambda w: CI(w, df_emission_scores['Carbon Intensity']) <= 0.8*benchmark_emissions)
weights = ef_constrained2.convex_objective(TE, portfolio_returns=df_sample, benchmark_returns=benchmark_returns)

In [71]:
constrained2_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
constrained2_weights[constrained2_weights['Weight']>10**-6].sort_values(by='Weight', ascending=False)

,Weight
WMT,0.153755
ACN,0.088461
ABT,0.067916
ADBE.O,0.064073
QCOM.O,0.048141
...,...
TPIC.O,0.000362
INFN.O,0.000362
BOOM.O,0.000335
GOGL.O,0.000297


In [72]:
constrained2_returns = (constrained2_weights.T.values*df_sample).sum(axis=1)

In [73]:
mu_constrained2_monthly = constrained2_returns.mean()
mu_constrained2_annual = (1+mu_constrained2_monthly)**12-1

sigma_sq_constrained2 =constrained2_returns.var(ddof=1)
sigma_constrained2_monthly = constrained2_returns.std(ddof=1)
sigma_constrained2_annual = sigma_constrained2_monthly*np.sqrt(12)

sr_constrained2 = mu_constrained2_annual/sigma_constrained2_annual

num_inv_constrained2 = np.sum(constrained2_weights.values>10**-6)
num_sectors_constrained2 = df_sector.merge(constrained2_weights.loc[constrained2_weights['Weight']>10**-6], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

In [74]:
constrained2_emissions = (constrained2_weights['Weight'].values*df_emission_scores.drop(columns=['RIC']).T).sum(1)

#### Constrained Portfolio - 50% Reduction in Carbon Intensity

In [86]:
ef_constrained3 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained3.add_constraint(lambda w: TE(w, df_sample, benchmark_returns) <= 0.1**2)
ef_constrained3.add_constraint(lambda w: CI(w, df_emission_scores['Carbon Intensity']) <= 0.5*benchmark_emissions)
weights = ef_constrained3.convex_objective(TE, portfolio_returns=df_sample, benchmark_returns=benchmark_returns)

In [101]:
constrained3_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
constrained3_weights[constrained3_weights['Weight']>10**-4].sort_values(by='Weight', ascending=False).head(15)

,Weight
OTIS.K,0.229722
CHK.O,0.167951
PGR,0.140532
BAH,0.134186
RLI,0.073845
CTVA.K,0.065569
PWR,0.056048
RGEN.O,0.040936
ETSY.O,0.037909
WK,0.026605


In [91]:
constrained3_returns = (constrained3_weights.T.values*df_sample).sum(axis=1)

mu_constrained3_monthly = constrained3_returns.mean()
mu_constrained3_annual = (1+mu_constrained3_monthly)**12-1

sigma_sq_constrained3 =constrained3_returns.var(ddof=1)
sigma_constrained3_monthly = constrained3_returns.std(ddof=1)
sigma_constrained3_annual = sigma_constrained3_monthly*np.sqrt(12)

sr_constrained3 = mu_constrained3_annual/sigma_constrained3_annual

num_inv_constrained3 = np.sum(constrained3_weights.values>10**-4)
num_sectors_constrained3 = df_sector.merge(constrained3_weights.loc[constrained3_weights['Weight']>10**-4], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

In [93]:
constrained3_emissions = (constrained3_weights['Weight'].values*df_emission_scores.drop(columns=['RIC']).T).sum(1)

#### Contrained Portfolio - Tangency Portfolio with 20% less Carbon Intensity

In [99]:
ef_constrained4 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained4.add_constraint(lambda w: CI(w, df_emission_scores['Carbon Intensity']) <= 0.8*benchmark_emissions)
weights = ef_constrained4.max_sharpe()

In [108]:
constrained4_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
constrained4_weights[constrained4_weights['Weight']>10**-4].sort_values(by='Weight', ascending=False).head(15)

,Weight
OTIS.K,0.229722
CHK.O,0.167951
PGR,0.140532
BAH,0.134186
RLI,0.073845
CTVA.K,0.065569
PWR,0.056048
RGEN.O,0.040936
ETSY.O,0.037909
WK,0.026605


In [104]:
constrained4_returns = (constrained4_weights.T.values*df_sample).sum(axis=1)

mu_constrained4_monthly = constrained4_returns.mean()
mu_constrained4_annual = (1+mu_constrained4_monthly)**12-1

sigma_sq_constrained4 = constrained4_returns.var(ddof=1)
sigma_constrained4_monthly = constrained4_returns.std(ddof=1)
sigma_constrained4_annual = sigma_constrained4_monthly*np.sqrt(12)

sr_constrained4 = mu_constrained4_annual/sigma_constrained4_annual

num_inv_constrained4 = np.sum(constrained4_weights.values>10**-4)
num_sectors_constrained4 = df_sector.merge(constrained4_weights.loc[constrained4_weights['Weight']>10**-4], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()